<a href="https://colab.research.google.com/github/atik-bd/MSc-BA/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data Preparation
from pandas import read_csv, get_dummies,Series,DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
data=read_csv('/content/drive/MyDrive/DataSets/Emloyees.csv')# reading
data['PastEmployee'] = data['PastEmployee'].map({'Yes':1, 'No':0}) # encoding output
data['OverTime'] = data['OverTime'].map({'Yes':1, 'No':0}) # encoding
data['Gender'] = data['Gender'].map({'Female':1, 'Male':0}) # encoding
data2 = get_dummies(data, columns =  ['BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus']) # encoding
data2.info()
X = data2.drop('PastEmployee', axis = 1) # Features
Y = data2['PastEmployee'] # Labels
X_scaled = StandardScaler().fit_transform(X) # scaling
X_train, X_test, Y_train, Y_test = train_test_split( X_scaled, Y, test_size = 0.2, random_state = 100)# splitting
X_train,Y_train =SMOTE (random_state = 100).fit_resample(X_train,Y_train)# balancing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 32 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                1470 non-null   int64
 1   PastEmployee                       1470 non-null   int64
 2   DistanceFromHome                   1470 non-null   int64
 3   EnvironmentSatisfaction            1470 non-null   int64
 4   Gender                             1470 non-null   int64
 5   MonthlyIncome                      1470 non-null   int64
 6   NumCompaniesWorked                 1470 non-null   int64
 7   OverTime                           1470 non-null   int64
 8   BusinessTravel_Non-Travel          1470 non-null   bool 
 9   BusinessTravel_Travel_Frequently   1470 non-null   bool 
 10  BusinessTravel_Travel_Rarely       1470 non-null   bool 
 11  Department_Human Resources         1470 non-null   bool 
 12  Department_Research 

In [ ]:
# Decision Tree Classifier (method#1)
from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier as dt
# import sklearn as  sk
# import sklearn
#DT_classifier =sk.tree.DecisionTreeClassifier()# classifier building
DT_classifier1 =tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 5)# classifier building
DT_classifier1.fit(X_train, Y_train) # training
Y_pred1 = DT_classifier1.predict(X_test) # testing
# # imp_features =Series(DT_classifier.feature_importances_, index=list(X)).sort_values(ascending=False) # what are the important features?
# # print(imp_features)

In [ ]:
# Evaluation
# Acuracy and confusion matrix
from sklearn import metrics
Accuracy=metrics.accuracy_score(Y_test, Y_pred1) # calculating accuaracy
print("Accuracy: ", Accuracy) # Is this a good metric??
con_matrix = metrics.confusion_matrix(Y_test, Y_pred1)
print (con_matrix)
recall = metrics.recall_score(Y_test, Y_pred1)
print ("recall :",recall)
percision=metrics.precision_score(Y_test, Y_pred1)
print("percesion :",percision)
f1=metrics.f1_score(Y_test,Y_pred1)
print('f1:',f1)

Accuracy:  0.8503401360544217
[[225  21]
 [ 23  25]]
recall : 0.5208333333333334
percesion : 0.5434782608695652
f1: 0.5319148936170213


In [ ]:
#using GridSearch (method#2)
from sklearn.model_selection import GridSearchCV
DT_classifier2 = tree.DecisionTreeClassifier(criterion = 'entropy') # building classfier
depth = {'max_depth': [2,3,4,5,10,15,20,25,30,35]}
grid_search1 = GridSearchCV(estimator=DT_classifier2, param_grid=depth, scoring='precision', cv=5)# building
grid_search1.fit(X_scaled, Y )# training, testing , evaluation, ranking.
best_depth = grid_search1.best_params_
print(best_depth)
best_result = grid_search1.best_score_
print(best_result)

{'max_depth': 3}
0.5916666666666666


In [ ]:
# Decision Tree Classifier (method#1) with th best depth
from sklearn import tree
DT_classifier3 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 25)# classifier building
DT_classifier3.fit(X_train, Y_train) # training
Y_pred2 = DT_classifier3.predict(X_test) # testing
imp_features = Series(DT_classifier3.feature_importances_, index=list(X)).sort_values(ascending=False) # what are the important features?
print(imp_features)

OverTime                             0.238191
MonthlyIncome                        0.148355
MaritalStatus_Single                 0.110810
EnvironmentSatisfaction              0.096847
Age                                  0.092591
NumCompaniesWorked                   0.055425
DistanceFromHome                     0.049838
JobRole_Sales Executive              0.025043
Department_Research & Development    0.022439
BusinessTravel_Travel_Frequently     0.021873
JobRole_Research Scientist           0.019683
EducationField_Life Sciences         0.019663
EducationField_Technical Degree      0.012569
BusinessTravel_Travel_Rarely         0.011185
Gender                               0.011139
EducationField_Marketing             0.011068
EducationField_Medical               0.009547
MaritalStatus_Married                0.009528
JobRole_Laboratory Technician        0.007782
BusinessTravel_Non-Travel            0.007323
JobRole_Sales Representative         0.005549
EducationField_Other              

In [ ]:
#using Grid Search (method#2) with best feature
X2 = data2[['OverTime','MonthlyIncome','MaritalStatus_Single','EnvironmentSatisfaction','Age']] # Features#
#Y = data2['PastEmployee'] # Labels
X_scaled = StandardScaler().fit_transform(X2) # scaling
# X_train, X_test, Y_train, Y_test = train_test_split( X_scaled, Y, test_size = 0.3, random_state = 100)# splitting
# X_train,Y_train =SMOTE (random_state = 100).fit_resample(X_train,Y_train)# balancing

DT_classifier4 = tree.DecisionTreeClassifier(criterion = 'entropy') # building classfier
depth = {'max_depth': [2,3,4,5,10,15,20,25,30,35]}
grid_search2 = GridSearchCV(estimator=DT_classifier4, param_grid=depth, scoring='recall', cv=5)
grid_search2.fit(X_scaled, Y) # Training, testing , evaluation, ranking.
best_depth = grid_search2.best_params_
print(best_depth)
best_result = grid_search2.best_score_
print(best_result)

{'max_depth': 25}
0.33758865248226955


In [ ]:
# Decision Tree Classifier (method#1) with the best depth and best features
X2 = data2[['OverTime','MonthlyIncome','MaritalStatus_Single','EnvironmentSatisfaction','Age']] # Features
#Y = data2['PastEmployee'] # Labels
X_scaled = StandardScaler().fit_transform(X2) # scaling
X_train, X_test, Y_train, Y_test = train_test_split( X_scaled, Y, test_size = 0.3, random_state = 100)# splitting
X_train,Y_train =SMOTE (random_state = 100).fit_resample(X_train,Y_train)# balancing
DT_classifier5 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 3)# classifier building
DT_classifier5.fit(X_train, Y_train) # training
Y_pred3 = DT_classifier5.predict(X_test) # testing
# imp_features = Series(DT_classifier5.feature_importances_, index=list(X)).sort_values(ascending=False) # what are the important features?
# print(imp_features)
# Acuracy and confusion matrix
from sklearn import metrics
Accuracy=metrics.accuracy_score(Y_test, Y_pred3) # calculating accuaracy
print("Accuracy: ", Accuracy) # Is this a good metric??
con_matrix = metrics.confusion_matrix(Y_test, Y_pred3)
print (con_matrix)
recall = metrics.recall_score(Y_test, Y_pred3)
print (recall)
percision=metrics.precision_score(Y_test, Y_pred3)
print(percision)

Accuracy:  0.7324263038548753
[[283  91]
 [ 27  40]]
0.5970149253731343
0.3053435114503817
